# Day 3: Crossed Wires

https://adventofcode.com/2019/day/3

Two **wires** are connected to a **central port** and extend outward on a **grid**. 

You trace the path each wire takes as it leaves the central port, one wire per line of text.

## Commands

If the first wire's path is `R8,U5,L5,D3`, then starting from the central port (o), 

it goes right 8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

In [348]:
module Command =
    
    type Distance = int
    
    type Direction =
        | U
        | D
        | L
        | R
        static member ofChar c : Direction =
            match c with
            | 'U' -> U
            | 'D' -> D
            | 'L' -> L
            | 'R' -> R
            | X -> failwith (sprintf "Invalid direction %A" X) 
    
    type Command = 
        | Command of Direction * Distance
        static member ofString (s:String) : Command = 
            
            let intFromCharSeq s = s |> Array.ofSeq |> String |> Int32.Parse
            
            let (dir,dist) = 
                match s |> Seq.toList with
                | dir::dist -> (Direction.ofChar dir, intFromCharSeq dist)
                |  _ -> failwith (sprintf "Invalid command string %A" s)  
                    
            Command ( dir , dist )
    
    let parse (input:String) : Command seq =
        input.Split(",") 
        |> Seq.ofArray
        |> Seq.map Command.ofString


open Command

"R8,U5,L5,D3" |> Command.parse |> Seq.map (string)

index,value
0,"Command (R,8)"
1,"Command (U,5)"
2,"Command (L,5)"
3,"Command (D,3)"


## Points

To fix the circuit, you need to find the intersection point closest to the central port. 

Because the wires are on a grid, use the [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry) for this measurement. 

These wires cross at two locations (Edit: _3,3 and 6,6_), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

In [349]:
module Point=
    
    type Point = 
        | Point of int*int
        static member distanceTo (Point (x1,y1)) (Point (x2,y2)) : int =  abs(x1-x2) + abs(y1-y2)
        static member distance (p:Point) : int = Point.distanceTo p (Point (0,0))

open Point 

Point (3,3) |> Point.distance

6

## Wires

The wires twist and turn, but the two wires occasionally cross paths. 

While the wires do technically cross right at the central port where they both start, 

this point does not count, nor does a wire count as crossing with itself.

> Since we are working on a grid, we can consider a wire a collection of points.
>
> The intersection of two wires are the points they have in common.
>
> - To compute all the `Point`s that make a `Wire`, we will use a `Cursor` that will 
follow the provided `Commands`.
> - In order to simplify the `Cursor` behavior, the `Commands` will expanded to perform single steps.

In [350]:
open Command

// extend Command definition

type Command with
    // return a sequence of single steps (as Direction)
    static member AsDirections (c:Command) : Direction seq =
        let (Command (dir, dist)) = c
        seq { for i in 1 .. dist -> dir }

module Cursor = 
    
    open Command
    open Point
    
    type Cursor(p:Point) =
        let mutable position = p
        member self.Move (d:Direction) =
            let (Point (x,y)) = position
            match d with
            | U -> position <- Point(x,y+1)
            | D -> position <- Point(x,y-1)
            | L -> position <- Point(x-1,y)
            | R -> position <- Point(x+1,y)
        member self.Position with get() = position

module Wire =
    
    open Command
    open Point
    open Cursor
    
    type Wire = 
        | Wire of Command seq
        
        static member ofString s : Wire = Wire (Command.parse s)
            
        static member getPoints (Wire commands) : Point seq =
    
            // create a cursor at the start position

            let cursor = Cursor (Point (0,0))

            seq { // return a sequence ...

                yield cursor.Position // ... with first item being the "central point"

                for command in commands do  // .. followed by
                    
                    // .. sequence of points each command makes the cursor visit
                    yield!  command
                            |> Command.AsDirections // convert to multiple single steps (as a sequence of Direction)
                            |> Seq.map ( fun d -> 
                                            do cursor.Move d   // move cursor one step as specified direction
                                            cursor.Position    // return the position the cursor it's at
                            ) //each position the cursor visits
            }


# Example

If the first wire's path is `R8,U5,L5,D3`, then starting from the central port (o), it lands at point **(3,2)**.

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
2...X....|.
.........|.
.o-------+.
....3......
```


In [351]:
open Wire
"R8,U5,L5,D3" |> Wire.ofString |> Wire.getPoints |> Seq.rev |> Seq.head |> (string)

Point (3,2)

Then, if the second wire's path is `U7,R6,D4,L4`, it goes up 7, right 6, down 4, and left 4,
landing at **(2,3)**

```
...........
.+-----+...
.|.....|...
.|.....|...
.|.....|...
3|.X---+...
.|.........
.|.........
.o.........
...2.......

```

In [352]:
open Wire
"U7,R6,D4,L4" |> Wire.ofString |> Wire.getPoints |> Seq.rev |> Seq.head

Tag,Item1,Item2
0,2,3


## Intersect

To find the intersection points, we just need to find common points in both wires. 

For that we will use [Set.intersect](https://msdn.microsoft.com/visualfsharpdocs/conceptual/set.intersect%5b%27t%5d-function-%5bfsharp%5d), after converting sequence of points to sets. 

We must exclude the root node `(0,0)`, which in our case is the **first** of the sequence of points.

In [353]:
open Wire

type Wire with

    static member intersect (w1:Wire) (w2:Wire) : Set<Point> =
    
        let points_w1 = w1 |> Wire.getPoints |> Set.ofSeq
        let points_w2 = w2 |> Wire.getPoints |> Set.ofSeq

        Set.intersect points_w1 points_w2
        
Wire.intersect (Wire.ofString "R8,U5,L5,D3") (Wire.ofString "U7,R6,D4,L4") |> (string)

set [Point (0,0); Point (3,3); Point (6,5)]

These wires cross at two locations (marked X), but the lower-left one 

is closer to the central port: **its distance is 3 + 3 = 6.**

In [354]:
open Wire
open Point

type Wire with
    
    static member intersection (w1:Wire) (w2:Wire) : int =
        
        Wire.intersect w1 w2 
        |> Set.map Point.distance
        |> Seq.sort
        |> Seq.tail // skip central point (0,0), which has distance 0
        |> Seq.head // take next point with lowest distance

Wire.intersection (Wire.ofString "R8,U5,L5,D3") (Wire.ofString "U7,R6,D4,L4") |> (string)

6

## Test cases

`R75,D30,R83,U83,L12,D49,R71,U7,L72`

`U62,R66,U55,R34,D71,R55,D58,R83`

= distance 159

In [355]:
open Wire

( Wire.ofString "R75,D30,R83,U83,L12,D49,R71,U7,L72" ,
  Wire.ofString "U62,R66,U55,R34,D71,R55,D58,R83"
) ||> Wire.intersection |> (string)

159

`R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51`

`U98,R91,D20,R16,D67,R40,U7,R15,U6,R7`

= distance 135

In [356]:
open Wire

( Wire.ofString "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51" ,
  Wire.ofString "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"
) ||> Wire.intersection |> (string)

135

## Input

The input is provided as two lines of strings, representing the two wires.


In [357]:
let inputFilePath = "./03-input-1.txt"

let input = System.IO.File.ReadAllLines(inputFilePath)
let wire1 = input.[0] |> Wire.ofString
let wire2 = input.[1] |> Wire.ofString

## Solution

In [358]:
#r "nuget:Expecto"
    
open Expecto

try 
    let result = Wire.intersection wire1 wire2
    let expected = 217
    Expect.equal result expected (sprintf "Unexpected result: %A (Should be %A)" result expected)
    "That's the right answer! You are one gold star closer to rescuing Santa."
with
    | ex -> ex.ToString()

That's the right answer! You are one gold star closer to rescuing Santa.